# Data wrapper

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

def extrair_detalhes_pet(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Extrair detalhes do pet
    nome_pet = soup.find('div', class_='v-col-md-8').find('span').get_text(strip=True)
    
    status_pet_tag = soup.find('div', class_='v-chip__content')
    status_pet = status_pet_tag.get_text(strip=True) if status_pet_tag else 'Não encontrado'
    
    descricao_pet_tag = soup.find('div', class_='v-alert__content')
    descricao_pet = descricao_pet_tag.get_text(strip=True) if descricao_pet_tag else 'Não encontrado'
    
    abrigo_presente, nome_abrigo, endereco_abrigo = verificar_abrigo(page_html)
    
    url_imagem = extrair_url_imagem(url)
    
    return {'Nome do Pet': nome_pet, 'Status do Pet': status_pet, 'Descrição do Pet': descricao_pet, 
            'Abrigo': nome_abrigo if abrigo_presente else 'Não encontrado', 'Endereço do Abrigo': endereco_abrigo if abrigo_presente else 'Não encontrado', 
            'URL': url, 'URL da Imagem': url_imagem}

def extrair_url_imagem(url):
    # Configuração do Selenium para usar um driver do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Para executar o Chrome em modo headless (sem interface gráfica)
    driver = webdriver.Chrome(options=options)
    
    # Carregar a página
    driver.get(url)
    
    # Obter o código HTML da página após a renderização completa
    page_html = driver.page_source
    
    # Fechar o driver do Selenium
    driver.quit()
    
    # Parsear o HTML com BeautifulSoup
    soup = BeautifulSoup(page_html, 'html.parser')
    
    # Encontrar a tag img com a classe v-img__img e extrair a URL da imagem
    img_tag = soup.find('img', class_='v-img__img')
    if img_tag:
        return img_tag['src']
    else:
        return None

def verificar_abrigo(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    abrigo_info = soup.find('div', class_='v-list-item-title', text=lambda text: text and text.startswith('Abrigo:'))
    if abrigo_info:
        abrigo = abrigo_info.text.strip().replace('Abrigo:', '').strip()
        endereco = abrigo_info.find_next('div', class_='v-list-item-subtitle').text.strip()
        return True, abrigo, endereco
    
    return False, None, None

# Lista para armazenar os dados de cada pet
dados_pets = []

# Varrendo os pets
for pet_id in range(10, 2710):
    url_pet = f'https://petsrs.com.br/pet/{pet_id}'
    detalhes_pet = extrair_detalhes_pet(url_pet)
    dados_pets.append(detalhes_pet)

# Criando DataFrame com os dados dos pets
df_pets = pd.DataFrame(dados_pets)

print(df_pets)


C:\Users\Pichau\AppData\Local\Temp\ipykernel_27484\2439045566.py:68: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  abrigo_info = soup.find('div', class_='v-list-item-title', text=lambda text: text and text.startswith('Abrigo:'))


       Nome do Pet    Status do Pet  \
0           Barthô     Desaparecido   
1  Procura-se dono  Procura-se dono   

                                    Descrição do Pet          Abrigo  \
0  Preto e branco, castrado, em torno de 1 ano e ...  Não encontrado   
1  @petsresgatadosctg3ulbra\nGALPÃO ULBRA\nPASTOR...           ULBRA   

          Endereço do Abrigo                             URL  \
0             Não encontrado  https://petsrs.com.br/pet/1838   
1  Avenida Farroupilha 8001.  https://petsrs.com.br/pet/1839   

                                       URL da Imagem  
0                                               None  
1  https://d3grcwqt8cmtlx.cloudfront.net/Screensh...  


In [3]:
df_pets.head()

,Nome do Pet,Status do Pet,Descrição do Pet,Abrigo,Endereço do Abrigo,URL,URL da Imagem
0,Barthô,Desaparecido,"Preto e branco, castrado, em torno de 1 ano e ...",Não encontrado,Não encontrado,https://petsrs.com.br/pet/1838,None
1,Procura-se dono,Procura-se dono,@petsresgatadosctg3ulbra\nGALPÃO ULBRA\nPASTOR...,ULBRA,Avenida Farroupilha 8001.,https://petsrs.com.br/pet/1839,https://d3grcwqt8cmtlx.cloudfront.net/Screensh...


In [ ]:
# Salvando o DataFrame em um arquivo Excel
df_pets.to_excel('database.xlsx', index=False)